In [63]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [37]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\kunaual\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


## Get article titles and teaser text 

In [3]:
url = 'https://redplanetscience.com/'
browser.visit(url)

In [30]:
html = browser.html  #get all the html
soup = BeautifulSoup(html, 'html.parser')

#testing output, see what to get
#print(soup.prettify())
 

titles=[]
teaser_texts=[]

divs = soup.find_all('div')
for div in divs:
    #print(div['class'][0])  #this will get you content_title and article_teaser_body
    if div['class'][0]=='content_title':
        #print(div.text)
        titles.append(div.text)
    if div['class'][0]=='article_teaser_body':
        #print(div.text)
        teaser_texts.append(div.text)



## Get featured image url

In [38]:
iurl = 'https://spaceimages-mars.com/'
browser.visit(iurl)

In [61]:
ihtml = browser.html
isoup = BeautifulSoup(ihtml, 'html.parser')

#checkout html output, looking for featured image 
#print(isoup.prettify())

# looking for this src:
# <img class="headerimage fade-in" src="image/featured/mars1.jpg">

imgs = isoup.find_all('img')
for img in imgs:
    #print(img) #testing message
    try:
        if img['class'][0]=='headerimage':
            #print(img['src'])  #testing message
            featured_image_file = img['src']
            break   #header image will be towards the top, once found don't need to continue the loop
    except:  #if there's no class, continue to the next one
        continue

#concat the url to the image location
featured_image_url = iurl+featured_image_file

In [62]:
#check the url value
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

## Get Mars facts

In [65]:
turl = 'https://galaxyfacts-mars.com/'

In [72]:
tables = pd.read_html(turl)
tables[1]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [100]:
mars_earth_df = tables[0]
mars_facts_df = tables[1]

In [101]:
mars_facts_df.rename(columns={0:"FACT_DESC",1:"FACT_VALUE"}, inplace=True)

In [102]:
mars_earth_df.rename(columns={0:"ATTRIBUTE",1:"MARS",2:"EARTH"}, inplace=True)
mars_earth_df

,ATTRIBUTE,MARS,EARTH
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [104]:
#delete the 1st row which is more of a table header
mars_earth_df =mars_earth_df.drop(0)  
mars_earth_df  #not bothering to reindex because not keeping it in the html

,ATTRIBUTE,MARS,EARTH
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [106]:
#push dfs to html files
mars_earth_df.to_html('mars_earth_comp.html', index=False)
mars_facts_df.to_html('mars_facts.html', index=False)

## Get hemisphere info

In [182]:
hurl = 'https://marshemispheres.com/'
browser.visit(hurl)
hemisphere_image_urls = []

In [183]:
html = browser.html  #get all the html
hsoup = BeautifulSoup(html, 'html.parser')

In [200]:
#need to get the list of 4 headers from main page and then loop through them clicking next.
list_of_subpages = []

a_itemlinks = hsoup.find_all('a', class_='itemLink')
#h3s = hsoup.find_all('h3')
for alink in a_itemlinks:
    #print(alink['href']) #testing output
    cur_href = alink['href']
    if 'html' in cur_href:
        #print('it does') #testing output
        if cur_href not in list_of_subpages:
            #print('add it!') #testing output
            list_of_subpages.append(cur_href)

list_of_subpages

['cerberus.html', 'schiaparelli.html', 'syrtis.html', 'valles.html']

In [201]:
for page in list_of_subpages:
    sub_url=hurl+page
    print(sub_url)
    
    browser.visit(sub_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    
    
    #browser.links.find_by_partial_text('Back').click()
    

https://marshemispheres.com/cerberus.html
https://marshemispheres.com/schiaparelli.html
https://marshemispheres.com/syrtis.html
https://marshemispheres.com/valles.html


In [197]:
#testing output, see what to get
#print(hsoup.prettify())

#print(a_itemlinks.find('img'))
#img_info=a_itemlinks.find('img')

#this is getting the thumbnail off the main page
# hemis_dict = {}
# hemis_dict['title'] = img_info['alt'].split(" Enhanced")[0]
# hemis_dict['img_url'] = hurl+img_info['src']
# img_info['alt'].split(" Enhanced")[0]  #title!  yes, there are other ways to get this. could take if off 1st page too
# hurl+img_info['src']
#browser.links.find_by_partial_text('Back').click()

--------------------------------------------------------------
cerberus.html
it does
add it!
cerberus.html
it does
schiaparelli.html
it does
add it!
schiaparelli.html
it does
syrtis.html
it does
add it!
syrtis.html
it does
valles.html
it does
add it!
valles.html
it does
#


In [187]:
hemisphere_image_urls.append(hemis_dict)



'Cerberus Hemisphere'

In [179]:
for x in range(1):
    #browser.links.find_by_partial_text('Hemisphere').click()  #partial text needs to be a variable from loop
    browser.links.find_by_partial_text('Schiaparelli').click() 
    lhtml = browser.html  #get all the html
    lsoup = BeautifulSoup(lhtml, 'html.parser')
    #testing output, see what to get
    print(lsoup.prettify())
    print('--------------------------------------------------------------')
    a_itemlinks = lsoup.find('a', class_='itemLink')
    #h3s = hsoup.find_all('h3')
    print(a_itemlinks.find('img'))
    img_info=a_itemlinks.find('img')
    print('fucking'+img_info['alt'])
    hemis_dict = {}
    hemis_dict['title'] = img_info['alt'].split(" Enhanced")[0]
    hemis_dict['img_url'] = hurl+img_info['src']
    #img_info['alt'].split(" Enhanced")[0]  #title!  yes, there are other ways to get this. could take if off 1st page too
    #hurl+img_info['src']
    hemisphere_image_urls.append(hemis_dict)
    
    
    #browser.links.find_by_partial_text('Back').click() 

<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <title>
   Astropedia Search Results | GUSS Astrogeology Science Center
  </title>
  <meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
  <meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <link href="css/main.css" media="screen" rel="stylesheet"/>
  <link href="css/print.css" media="print" rel="stylesheet"/>
  <link href="#" rel="icon" type="image/x-ico"/>
 </head>
 <body id="results">
  <header>
   <a href="#" style="float:right;margin-top:10px;" target="_blank">
    <img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_

TypeError: 'NoneType' object is not subscriptable

In [181]:
hemis_dict = {}
hemis_dict['title'] = img_info['alt'].split(" Enhanced")[0]
hemis_dict['img_url'] = hurl+img_info['src']
#img_info['alt'].split(" Enhanced")[0]  #title!  yes, there are other ways to get this. could take if off 1st page too
#hurl+img_info['src']
hemisphere_image_urls.append(hemis_dict)

TypeError: 'NoneType' object is not subscriptable

In [180]:
hemis_dict
hemisphere_image_urls

[]

In [ ]:
try:
    for title_url in titles_and_urls:
        browser.links.find_by_partial_text('next').click()
except ElementDoesNotExist:
    print("Scraping Complete")

In [ ]:
browser.quit()